In [35]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import multiprocessing
import numpy as np
import yfinance as yf
import talib
import pandas as pd

In [61]:
data = yf.download('^NSEI', start='2020-01-01', end='2024-12-31', auto_adjust=True)
# Step 2: Fix columns
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

# Step 3: Prepare data
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.dropna(inplace=True)


[*********************100%***********************]  1 of 1 completed


In [10]:
data

Price,Open,High,Low,Close,Volume
Date,,,,,
2020-01-01,691.535677,695.681250,685.841253,687.708984,14004468
2020-01-02,688.802388,701.990786,688.802388,699.416870,17710316
2020-01-03,698.369044,702.309653,693.813481,700.259644,20984698
2020-01-06,692.446853,696.045712,682.424578,684.019043,24519177
2020-01-07,691.991251,699.052428,689.485697,694.542419,16683622
...,...,...,...,...,...
2024-12-23,1215.000000,1227.199951,1213.199951,1222.300049,10052824
2024-12-24,1222.300049,1233.550049,1221.000000,1222.750000,6734917
2024-12-26,1224.250000,1227.699951,1214.250000,1216.550049,10016178


In [62]:
def vwap_series(high, low, close, volume):
    typical_price = (high + low + close) / 3
    vwap = np.cumsum(typical_price * volume) / np.cumsum(volume)
    return vwap
#main strategy
class VWAP(Strategy):
    timeperiod = 14 
    def init(self):
        high = self.data.High
        low = self.data.Low
        close = self.data.Close
        volume = self.data.Volume

        self.vwap = self.I(vwap_series, high, low, close, volume)
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=20)
        self.upper, self.middle, self.lower = self.I(
            talib.BBANDS,
            self.data.Close,
            timeperiod=self.timeperiod,  # <--- Use self.timeperiod here
            nbdevup=2,
            nbdevdn=2,  
            matype=0
        )
    def next(self):
        price = self.data.Close[-1]
        vwap = self.vwap[-1]

       
        if self.rsi>55 or price>self.upper:
            self.sell()
        elif price<self.upper or price>self.middle:
            self.position.close()
            self.buy()
        elif price>self.lower or price < self.middle:
            self.position.close()
            self.buy()
        elif self.rsi<45 or price<self.lower:
            self.buy()
        
            
bt = Backtest(data, VWAP, cash=10_000)
stats= bt.run()
print(stats)
bt.plot()


C:\Users\risha\AppData\Local\Temp\ipykernel_19384\3704500246.py:43: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, VWAP, cash=10_000)
C:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=22: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=23: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\risha\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=24: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\risha\AppData\Local\Programs\Python\Py

Start                     2020-01-01 00:00:00
End                       2024-12-30 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    95.95469
Equity Final [$]                  24537.30078
Equity Peak [$]                   27108.45117
Return [%]                          145.37301
Buy & Hold Return [%]                94.93714
Return (Ann.) [%]                    20.08237
Volatility (Ann.) [%]                21.03637
CAGR [%]                             13.19523
Sharpe Ratio                          0.95465
Sortino Ratio                          1.6317
Calmar Ratio                            0.929
Alpha [%]                            63.75504
Beta                                  0.85971
Max. Drawdown [%]                   -21.61722
Avg. Drawdown [%]                     -2.1673
Max. Drawdown Duration      402 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                  533
Win Rate [%]                      

GridPlot(id='p11722', ...)

In [58]:
# try to implement more indicators but the ouput was not better
def vwap_series(high, low, close, volume):
    typical_price = (high + low + close) / 3
    vwap = np.cumsum(typical_price * volume) / np.cumsum(volume)
    return vwap

class VWAPEnhanced(Strategy):
    timeperiod = 14
    def init(self):
        close = self.data.Close.astype(np.float64)
        high = self.data.High.astype(np.float64)
        low = self.data.Low.astype(np.float64)
        volume = self.data.Volume.astype(np.float64)

        self.vwap = self.I(vwap_series, high, low, close, volume)
        self.rsi = self.I(talib.RSI, close, timeperiod=20)
        self.upper, self.middle, self.lower = self.I(
            lambda x: talib.BBANDS(x, timeperiod=self.timeperiod, nbdevup=2, nbdevdn=2, matype=0),
            close
        )
        self.ema200 = self.I(talib.EMA, close, timeperiod=200)
        self.macdhist = self.I(lambda x: talib.MACD(x)[2], close)
        self.atr = self.I(talib.ATR, high, low, close, timeperiod=14)
        self.vol_ma = self.I(lambda x: talib.SMA(x, timeperiod=20), volume)

    def next(self):
        if len(self.data.Close) < 200:
            return  

        price = self.data.Close[-1]
        rsi = self.rsi[-1]
        macdhist = self.macdhist[-1]
        upper = self.upper[-1]
        lower = self.lower[-1]
        middle = self.middle[-1]

        if not self.position: 
            if rsi > 70 or price > upper or macdhist > 0:
                self.buy()
           
            elif rsi < 30 or price < lower or macdhist < 0:
                self.sell()

        elif self.position.is_long:
          
            if rsi < 30 or price < lower or macdhist < 0:
                self.position.close()
                self.sell()

        elif self.position.is_short:
            
            if rsi > 70 or price > upper or macdhist > 0:
                self.position.close()
                self.buy()



bt = Backtest(data, VWAPEnhanced, cash=10_000, commission=0.001)
stats = bt.run()
print(stats)
bt.plot()


Start                     2020-01-01 00:00:00
End                       2024-12-30 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    83.18513
Equity Final [$]                   7840.03978
Equity Peak [$]                    17417.0844
Commissions [$]                    2375.89854
Return [%]                           -21.5996
Buy & Hold Return [%]               310.54329
Return (Ann.) [%]                    -4.83645
Volatility (Ann.) [%]                23.88654
CAGR [%]                             -3.30428
Sharpe Ratio                         -0.20248
Sortino Ratio                        -0.27454
Calmar Ratio                         -0.08206
Alpha [%]                           -29.47408
Beta                                  0.02536
Max. Drawdown [%]                   -58.93856
Avg. Drawdown [%]                   -10.87483
Max. Drawdown Duration      457 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                          

Start                     2020-01-01 00:00:00
End                       2024-12-30 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    83.18513
Equity Final [$]                   7840.03978
Equity Peak [$]                    17417.0844
Commissions [$]                    2375.89854
Return [%]                           -21.5996
Buy & Hold Return [%]               310.54329
Return (Ann.) [%]                    -4.83645
Volatility (Ann.) [%]                23.88654
CAGR [%]                             -3.30428
Sharpe Ratio                         -0.20248
Sortino Ratio                        -0.27454
Calmar Ratio                         -0.08206
Alpha [%]                           -29.47408
Beta                                  0.02536
Max. Drawdown [%]                   -58.93856
Avg. Drawdown [%]                   -10.87483
Max. Drawdown Duration      457 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                          

GridPlot(id='p10300', ...)